In [1]:
import argparse
import json
import os
import sys
from pathlib import Path
from threading import Thread

import numpy as np
import torch
from tqdm import tqdm



from models.experimental import attempt_load
from utils.callbacks import Callbacks
from utils.datasets import create_dataloader
from utils.general import (LOGGER, box_iou, check_dataset, check_img_size, check_requirements, check_suffix, check_yaml,
                           coco80_to_coco91_class, colorstr, increment_path, non_max_suppression, print_args,
                           scale_coords, xywh2xyxy, xyxy2xywh)
from utils.metrics import ConfusionMatrix, ap_per_class
from utils.plots import output_to_target, plot_images, plot_val_study
from utils.torch_utils import select_device, time_sync



## Code from VAL

In [2]:
__file__ = '/home/stud-dajo/tph-yolov5/val.py'

In [3]:
FILE = Path(__file__).resolve()
ROOT = FILE.parents[0]  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative


In [4]:
def save_one_txt(predn, save_conf, shape, file):
    # Save one txt result
    gn = torch.tensor(shape)[[1, 0, 1, 0]]  # normalization gain whwh
    for *xyxy, conf, cls in predn.tolist():
        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
        with open(file, 'a') as f:
            f.write(('%g ' * len(line)).rstrip() % line + '\n')


def save_one_json(predn, jdict, path, class_map):
    # Save one JSON result {"image_id": 42, "category_id": 18, "bbox": [258.15, 41.29, 348.26, 243.78], "score": 0.236}
    image_id = int(path.stem) if path.stem.isnumeric() else path.stem
    box = xyxy2xywh(predn[:, :4])  # xywh
    box[:, :2] -= box[:, 2:] / 2  # xy center to top-left corner
    for p, b in zip(predn.tolist(), box.tolist()):
        jdict.append({'image_id': image_id,
                      'category_id': class_map[int(p[5])],
                      'bbox': [round(x, 3) for x in b],
                      'score': round(p[4], 5)})


def process_batch(detections, labels, iouv):
    """
    Return correct predictions matrix. Both sets of boxes are in (x1, y1, x2, y2) format.
    Arguments:
        detections (Array[N, 6]), x1, y1, x2, y2, conf, class
        labels (Array[M, 5]), class, x1, y1, x2, y2
    Returns:
        correct (Array[N, 10]), for 10 IoU levels
    """
    correct = torch.zeros(detections.shape[0], iouv.shape[0], dtype=torch.bool, device=iouv.device)
    iou = box_iou(labels[:, 1:], detections[:, :4])
    x = torch.where((iou >= iouv[0]) & (labels[:, 0:1] == detections[:, 5]))  # IoU above threshold and classes match
    if x[0].shape[0]:
        matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()  # [label, detection, iou]
        if x[0].shape[0] > 1:
            matches = matches[matches[:, 2].argsort()[::-1]]
            matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
            # matches = matches[matches[:, 2].argsort()[::-1]]
            matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
        matches = torch.Tensor(matches).to(iouv.device)
        correct[matches[:, 1].long()] = matches[:, 2:3] >= iouv
    return correct


@torch.no_grad()
def run(data,
        weights=None,  # model.pt path(s)
        batch_size=32,  # batch size
        imgsz=640,  # inference size (pixels)
        conf_thres=0.001,  # confidence threshold
        iou_thres=0.6,  # NMS IoU threshold
        task='val',  # train, val, test, speed or study
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        single_cls=False,  # treat as single-class dataset
        augment=False,  # augmented inference
        verbose=False,  # verbose output
        save_txt=False,  # save results to *.txt
        save_hybrid=False,  # save label+prediction hybrid results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_json=False,  # save a COCO-JSON results file
        project=ROOT / 'runs/val',  # save to project/name
        name='exp',  # save to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        half=False,  # use FP16 half-precision inference
        model=None,
        dataloader=None,
        save_dir=Path(''),
        plots=True,
        callbacks=Callbacks(),
        compute_loss=None,
        ):
    # Initialize/load model and set device
    training = model is not None
    if training:  # called by train.py
        device = next(model.parameters()).device  # get model device

    else:  # called directly
        device = select_device(device, batch_size=batch_size)

        # Directories
        save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
        (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

        # Load model
        check_suffix(weights, '.pt')
        model = attempt_load(weights, map_location=device)  # load FP32 model
        gs = max(int(model.stride.max()), 32)  # grid size (max stride)
        imgsz = check_img_size(imgsz, s=gs)  # check image size

        # Multi-GPU disabled, incompatible with .half() https://github.com/ultralytics/yolov5/issues/99
        # if device.type != 'cpu' and torch.cuda.device_count() > 1:
        #     model = nn.DataParallel(model)

        # Data
        data = check_dataset(data)  # check

    # Half
    half &= device.type != 'cpu'  # half precision only supported on CUDA
    model.half() if half else model.float()

    # Configure
    model.eval()
    is_coco = isinstance(data.get('val'), str) and data['val'].endswith('coco/val2017.txt')  # COCO dataset
    nc = 1 if single_cls else int(data['nc'])  # number of classes
    iouv = torch.linspace(0.5, 0.95, 10).to(device)  # iou vector for mAP@0.5:0.95
    niou = iouv.numel()

    # Dataloader
    if not training:
        if device.type != 'cpu':
            model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
        pad = 0.0 if task == 'speed' else 0.5
        task = task if task in ('train', 'val', 'test') else 'val'  # path to train/val/test images
        dataloader = create_dataloader(data[task], imgsz, batch_size, gs, single_cls, pad=pad, rect=True,
                                       prefix=colorstr(f'{task}: '))[0]

    seen = 0
    confusion_matrix = ConfusionMatrix(nc=nc)
    names = {k: v for k, v in enumerate(model.names if hasattr(model, 'names') else model.module.names)}
    class_map = coco80_to_coco91_class() if is_coco else list(range(1000))
    s = ('%20s' + '%11s' * 6) % ('Class', 'Images', 'Labels', 'P', 'R', 'mAP@.5', 'mAP@.5:.95')
    dt, p, r, f1, mp, mr, map50, map = [0.0, 0.0, 0.0], 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    loss = torch.zeros(3, device=device)
    jdict, stats, ap, ap_class = [], [], [], []
    for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(dataloader, desc=s)):
        t1 = time_sync()
        img = img.to(device, non_blocking=True)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255  # 0 - 255 to 0.0 - 1.0
        targets = targets.to(device)
        nb, _, height, width = img.shape  # batch size, channels, height, width
        t2 = time_sync()
        dt[0] += t2 - t1

        # Run model
        out, train_out = model(img, augment=augment)  # inference and training outputs
        dt[1] += time_sync() - t2

        # Compute loss
        if compute_loss:
            loss += compute_loss([x.float() for x in train_out], targets)[1]  # box, obj, cls

        # Run NMS
        targets[:, 2:] *= torch.Tensor([width, height, width, height]).to(device)  # to pixels
        lb = [targets[targets[:, 0] == i, 1:] for i in range(nb)] if save_hybrid else []  # for autolabelling
        t3 = time_sync()
        out = non_max_suppression(out, conf_thres, iou_thres, labels=lb, multi_label=True, agnostic=single_cls)
        dt[2] += time_sync() - t3

        # Statistics per image
        for si, pred in enumerate(out):
            labels = targets[targets[:, 0] == si, 1:]
            nl = len(labels)
            tcls = labels[:, 0].tolist() if nl else []  # target class
            path, shape = Path(paths[si]), shapes[si][0]
            seen += 1

            if len(pred) == 0:
                if nl:
                    stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
                continue

            # Predictions
            if single_cls:
                pred[:, 5] = 0
            predn = pred.clone()
            scale_coords(img[si].shape[1:], predn[:, :4], shape, shapes[si][1])  # native-space pred

            # Evaluate
            if nl:
                tbox = xywh2xyxy(labels[:, 1:5])  # target boxes
                scale_coords(img[si].shape[1:], tbox, shape, shapes[si][1])  # native-space labels
                labelsn = torch.cat((labels[:, 0:1], tbox), 1)  # native-space labels
                correct = process_batch(predn, labelsn, iouv)
                if plots:
                    confusion_matrix.process_batch(predn, labelsn)
            else:
                correct = torch.zeros(pred.shape[0], niou, dtype=torch.bool)
            stats.append((correct.cpu(), pred[:, 4].cpu(), pred[:, 5].cpu(), tcls))  # (correct, conf, pcls, tcls)

            # Save/log
            if save_txt:
                save_one_txt(predn, save_conf, shape, file=save_dir / 'labels' / (path.stem + '.txt'))
            if save_json:
                save_one_json(predn, jdict, path, class_map)  # append to COCO-JSON dictionary
            callbacks.run('on_val_image_end', pred, predn, path, names, img[si])

        # Plot images
        if plots and batch_i < 3:
            f = save_dir / f'val_batch{batch_i}_labels.jpg'  # labels
            Thread(target=plot_images, args=(img, targets, paths, f, names), daemon=True).start()
            f = save_dir / f'val_batch{batch_i}_pred.jpg'  # predictions
            Thread(target=plot_images, args=(img, output_to_target(out), paths, f, names), daemon=True).start()

    # Compute statistics
    stats = [np.concatenate(x, 0) for x in zip(*stats)]  # to numpy
    if len(stats) and stats[0].any():
        p, r, ap, f1, ap_class = ap_per_class(*stats, plot=plots, save_dir=save_dir, names=names)
        ap50, ap = ap[:, 0], ap.mean(1)  # AP@0.5, AP@0.5:0.95
        mp, mr, map50, map = p.mean(), r.mean(), ap50.mean(), ap.mean()
        nt = np.bincount(stats[3].astype(np.int64), minlength=nc)  # number of targets per class
    else:
        nt = torch.zeros(1)

    # Print results
    pf = '%20s' + '%11i' * 2 + '%11.3g' * 4  # print format
    LOGGER.info(pf % ('all', seen, nt.sum(), mp, mr, map50, map))

    # Print results per class
    if (verbose or (nc < 50 and not training)) and nc > 1 and len(stats):
        for i, c in enumerate(ap_class):
            LOGGER.info(pf % (names[c], seen, nt[c], p[i], r[i], ap50[i], ap[i]))

    # Print speeds
    t = tuple(x / seen * 1E3 for x in dt)  # speeds per image
    if not training:
        shape = (batch_size, 3, imgsz, imgsz)
        LOGGER.info(f'Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {shape}' % t)

    # Plots
    if plots:
        confusion_matrix.plot(save_dir=save_dir, names=list(names.values()))
        callbacks.run('on_val_end')

    # Save JSON
    if save_json and len(jdict):
        w = Path(weights[0] if isinstance(weights, list) else weights).stem if weights is not None else ''  # weights
        anno_json = str(Path(data.get('path', '../coco')) / 'annotations/instances_val2017.json')  # annotations json
        pred_json = str(save_dir / f"{w}_predictions.json")  # predictions json
        LOGGER.info(f'\nEvaluating pycocotools mAP... saving {pred_json}...')
        with open(pred_json, 'w') as f:
            json.dump(jdict, f)

        try:  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoEvalDemo.ipynb
            check_requirements(['pycocotools'])
            from pycocotools.coco import COCO
            from pycocotools.cocoeval import COCOeval

            anno = COCO(anno_json)  # init annotations api
            pred = anno.loadRes(pred_json)  # init predictions api
            eval = COCOeval(anno, pred, 'bbox')
            if is_coco:
                eval.params.imgIds = [int(Path(x).stem) for x in dataloader.dataset.img_files]  # image IDs to evaluate
            eval.evaluate()
            eval.accumulate()
            eval.summarize()
            map, map50 = eval.stats[:2]  # update results (mAP@0.5:0.95, mAP@0.5)
        except Exception as e:
            LOGGER.info(f'pycocotools unable to run: {e}')

    # Return results
    model.float()  # for training
    if not training:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
    maps = np.zeros(nc) + map
    for i, c in enumerate(ap_class):
        maps[c] = ap[i]
    return (mp, mr, map50, map, *(loss.cpu() / len(dataloader)).tolist()), maps, t


def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data', type=str, default=ROOT / 'data/coco128.yaml', help='dataset.yaml path')
    parser.add_argument('--weights', nargs='+', type=str, default=ROOT / 'yolov5s.pt', help='model.pt path(s)')
    parser.add_argument('--batch-size', type=int, default=32, help='batch size')
    parser.add_argument('--imgsz', '--img', '--img-size', type=int, default=640, help='inference size (pixels)')
    parser.add_argument('--conf-thres', type=float, default=0.001, help='confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.6, help='NMS IoU threshold')
    parser.add_argument('--task', default='val', help='train, val, test, speed or study')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--single-cls', action='store_true', help='treat as single-class dataset')
    parser.add_argument('--augment', action='store_true', help='augmented inference')
    parser.add_argument('--verbose', action='store_true', help='report mAP by class')
    parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
    parser.add_argument('--save-hybrid', action='store_true', help='save label+prediction hybrid results to *.txt')
    parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
    parser.add_argument('--save-json', action='store_true', help='save a COCO-JSON results file')
    parser.add_argument('--project', default=ROOT / 'runs/val', help='save to project/name')
    parser.add_argument('--name', default='exp', help='save to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--half', action='store_true', help='use FP16 half-precision inference')
    opt = parser.parse_args()
    opt.data = check_yaml(opt.data)  # check YAML
    opt.save_json |= opt.data.endswith('coco.yaml')
    opt.save_txt |= opt.save_hybrid
    print_args(FILE.stem, opt)
    return opt

def parse_opt_1(data=None, weights=None, batch_size=None, imgsz=None, conf_thres=None,
              iou_thres=None, task=None, device=None, single_cls=None, augment=None,
              verbose=None, save_txt=None, save_hybrid=None, save_conf=None, save_json=None,
              project=None, name=None, exist_ok=None, half=None):
    parser = argparse.ArgumentParser()
    parser.add_argument('--data', type=str, default=data or (ROOT / 'data/coco128.yaml'), help='dataset.yaml path')
    parser.add_argument('--weights', nargs='+', type=str, default=weights or (ROOT / 'yolov5s.pt'), help='model.pt path(s)')
    parser.add_argument('--batch-size', type=int, default=batch_size or 32, help='batch size')
    parser.add_argument('--imgsz', '--img', '--img-size', type=int, default=imgsz or 640, help='inference size (pixels)')
    parser.add_argument('--conf-thres', type=float, default=conf_thres or 0.001, help='confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=iou_thres or 0.6, help='NMS IoU threshold')
    parser.add_argument('--task', default=task or 'val', help='train, val, test, speed or study')
    parser.add_argument('--device', default=device or '', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--single-cls', action='store_true' if single_cls else 'store_false', help='treat as single-class dataset')
    parser.add_argument('--augment', action='store_true' if augment else 'store_false', help='augmented inference')
    parser.add_argument('--verbose', action='store_true' if verbose else 'store_false', help='report mAP by class')
    parser.add_argument('--save-txt', action='store_true' if save_txt else 'store_false', help='save results to *.txt')
    parser.add_argument('--save-hybrid', action='store_true' if save_hybrid else 'store_false', help='save label+prediction hybrid results to *.txt')
    parser.add_argument('--save-conf', action='store_true' if save_conf else 'store_false', help='save confidences in --save-txt labels')
    parser.add_argument('--save-json', action='store_true' if save_json else 'store_false', help='save a COCO-JSON results file')
    parser.add_argument('--project', default=project or (ROOT / 'runs/val'), help='save to project/name')
    parser.add_argument('--name', default=name or 'exp', help='save to project/name')
    parser.add_argument('--exist-ok', action='store_true' if exist_ok else 'store_false', help='existing project/name ok, do not increment')
    parser.add_argument('--half', action='store_true' if half else 'store_false', help='use FP16 half-precision inference')
    
    opt = parser.parse_args([])
    opt.data = check_yaml(opt.data)  # check YAML
    opt.save_json |= opt.data.endswith('coco.yaml')
    opt.save_txt |= opt.save_hybrid
    print_args("FILE_STEM", opt)
    return opt


def main(opt):
    check_requirements(requirements=ROOT / 'requirements.txt', exclude=('tensorboard', 'thop'))

    if opt.task in ('train', 'val', 'test'):  # run normally
        run(**vars(opt))

    elif opt.task == 'speed':  # speed benchmarks
        # python val.py --task speed --data coco.yaml --batch 1 --weights yolov5n.pt yolov5s.pt...
        for w in opt.weights if isinstance(opt.weights, list) else [opt.weights]:
            run(opt.data, weights=w, batch_size=opt.batch_size, imgsz=opt.imgsz, conf_thres=.25, iou_thres=.45,
                device=opt.device, save_json=False, plots=False)

    elif opt.task == 'study':  # run over a range of settings and save/plot
        # python val.py --task study --data coco.yaml --iou 0.7 --weights yolov5n.pt yolov5s.pt...
        x = list(range(256, 1536 + 128, 128))  # x axis (image sizes)
        for w in opt.weights if isinstance(opt.weights, list) else [opt.weights]:
            f = f'study_{Path(opt.data).stem}_{Path(w).stem}.txt'  # filename to save to
            y = []  # y axis
            for i in x:  # img-sizey

                LOGGER.info(f'\nRunning {f} point {i}...')
                r, _, t = run(opt.data, weights=w, batch_size=opt.batch_size, imgsz=i, conf_thres=opt.conf_thres,
                              iou_thres=opt.iou_thres, device=opt.device, save_json=opt.save_json, plots=False)
                y.append(r + t)  # results and times
            np.savetxt(f, y, fmt='%10.4g')  # save
        os.system('zip -r study.zip study_*.txt')
        plot_val_study(x=x)  # plot

In [5]:
opt = parse_opt_1(
    weights='./weights/yolov5l-xs-1.pt',
    imgsz=1996,
    data='./data/VisDrone.yaml',
    augment=True,
    save_txt=True,
    save_conf=True,
    task='val',
    batch_size=1,
    verbose=True,
    name='v5l-xs'
)

FILE_STEM: data=./data/VisDrone.yaml, weights=./weights/yolov5l-xs-1.pt, batch_size=1, imgsz=1996, conf_thres=0.001, iou_thres=0.6, task=val, device=, single_cls=True, augment=False, verbose=False, save_txt=True, save_hybrid=True, save_conf=False, save_json=True, project=runs/val, name=v5l-xs, exist_ok=True, half=True


## CODE VAL End

In [6]:
opt

Namespace(data='./data/VisDrone.yaml', weights='./weights/yolov5l-xs-1.pt', batch_size=1, imgsz=1996, conf_thres=0.001, iou_thres=0.6, task='val', device='', single_cls=True, augment=False, verbose=False, save_txt=True, save_hybrid=True, save_conf=False, save_json=True, project=PosixPath('runs/val'), name='v5l-xs', exist_ok=True, half=True)

In [7]:
half=False
augment =True

In [8]:

device = select_device(opt.device, batch_size=opt.batch_size)


YOLOv5 🚀 052dfeb torch 2.1.1+cu121 CPU



In [9]:
model = attempt_load(opt.weights, map_location=device)  # load FP32 model

Fusing layers... 
Model Summary: 570 layers, 60425780 parameters, 0 gradients, 145.7 GFLOPs


In [10]:
model.float()

Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv3): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (m): Sequential(
        (0): Bottleneck(
          (cv1): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      

In [11]:
model.eval()

Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv3): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (m): Sequential(
        (0): Bottleneck(
          (cv1): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      

In [12]:
gs = max(int(model.stride.max()), 32)
nc = 10

In [13]:
iouv = torch.linspace(0.5, 0.95, 10).to(device)  # iou vector for mAP@0.5:0.95
niou = iouv.numel()

In [14]:
niou

10

In [15]:
pad = 0.5

In [16]:
task = 'val'

In [17]:
data = check_dataset(opt.data)

In [18]:
data

{'path': '../datasets/VisDrone',
 'train': '../datasets/VisDrone/VisDrone2019-DET-train/images',
 'val': '../datasets/VisDrone/VisDrone2019-DET-val/images',
 'test': '../datasets/VisDrone/VisDrone2019-DET-test-challenge/images',
 'nc': 10,
 'names': ['pedestrian',
  'people',
  'bicycle',
  'car',
  'van',
  'truck',
  'tricycle',
  'awning-tricycle',
  'bus',
  'motor'],
 'download': 'from utils.general import download, os, Path\n\ndef visdrone2yolo(dir):\n    from PIL import Image\n    from tqdm import tqdm\n\n    def convert_box(size, box):\n        # Convert VisDrone box to YOLO xywh box\n        dw = 1. / size[0]\n        dh = 1. / size[1]\n        return (box[0] + box[2] / 2) * dw, (box[1] + box[3] / 2) * dh, box[2] * dw, box[3] * dh\n\n    (dir / \'labels\').mkdir(parents=True, exist_ok=True)  # make labels directory\n    pbar = tqdm((dir / \'annotations\').glob(\'*.txt\'), desc=f\'Converting {dir}\')\n    for f in pbar:\n        img_size = Image.open((dir / \'images\' / f.name)

In [19]:
dataloader = create_dataloader(data[task], opt.imgsz, opt.batch_size, gs, opt.single_cls, pad=pad, rect=True,
                                       prefix=colorstr(f'{task}: '))[0]

val: Scanning '../datasets/VisDrone/VisDrone2019-DET-val/labels.cache' images and labels... 548 found, 0 missing, 0 empty, 0 corrupted: 100%


In [20]:
opt.data

'./data/VisDrone.yaml'

In [21]:
seen = 0
confusion_matrix = ConfusionMatrix(nc=nc)
names = {k: v for k, v in enumerate(model.names if hasattr(model, 'names') else model.module.names)}

In [22]:
names

{0: 'pedestrian',
 1: 'people',
 2: 'bicycle',
 3: 'car',
 4: 'van',
 5: 'truck',
 6: 'tricycle',
 7: 'awning-tricycle',
 8: 'bus',
 9: 'motor'}

In [23]:
is_coco = isinstance(data.get('val'), str) and data['val'].endswith('coco/val2017.txt')

In [24]:
is_coco

False

In [25]:
class_map = coco80_to_coco91_class() if is_coco else list(range(1000))

In [26]:
class_map

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [27]:
s = ('%20s' + '%11s' * 6) % ('Class', 'Images', 'Labels', 'P', 'R', 'mAP@.5', 'mAP@.5:.95')

In [28]:
s

'               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95'

In [29]:
dt, p, r, f1, mp, mr, map50, map = [0.0, 0.0, 0.0], 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
loss = torch.zeros(3, device=device)

In [30]:
jdict, stats, ap, ap_class = [], [], [], []

In [31]:
for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(dataloader, desc=s)):
    t1 = time_sync()
    img = img.to(device, non_blocking=True)
    
    img = img.half() if half else img.float()
    img = img / 255.0  # 0 - 255 to 0.0 - 1.0
    targets = targets.to(device)
    nb, _, height, width = img.shape 

    out, train_out = model(img, augment=augment)
    print(img)
    break

               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:   0%|                       | 0/548 [00:57<?, ?it/s]

tensor([[[[0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          ...,
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706]],

         [[0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          ...,
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706]],

         [[0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0

In [45]:
print(img.shape)

torch.Size([1, 3, 1152, 2016])


In [32]:
targets.shape

torch.Size([60, 6])

In [33]:
out.shape

torch.Size([1, 3070168, 15])

In [34]:
img.shape

torch.Size([1, 3, 1152, 2016])

In [35]:
targets[:, 2:] *= torch.Tensor([width, height, width, height]).to(device)  # to pixels

In [49]:
print(targets)

tensor([[   0.00000,    0.00000,  259.14084,  124.99976,   16.13554,   20.53371],
        [   0.00000,    0.00000,  265.74228,  112.53322,   20.53654,   39.59988],
        [   0.00000,    0.00000,  293.61444,  122.06686,   17.60388,   44.00035],
        [   0.00000,    0.00000, 1350.52356,  126.46621,   16.13550,   41.06633],
        [   0.00000,    0.00000, 1568.35962,  119.13281,   11.73462,   29.33356],
        [   0.00000,    0.00000, 1585.22925,  116.19991,   16.13562,   38.13342],
        [   0.00000,    0.00000, 1627.77063,  116.93370,   16.13550,   39.59987],
        [   0.00000,    0.00000, 1751.72400,  983.73364,   85.08081,  118.79968],
        [   0.00000,    0.00000,   30.30237,  964.66638,   39.60674,  139.33319],
        [   0.00000,    0.00000,   54.50770,  689.66638,   79.21346,  111.46625],
        [   0.00000,    0.00000,   91.17979,  680.86658,   35.20576,   82.13373],
        [   0.00000,    0.00000,  581.12787,  738.06616,   85.08081,  120.26721],
        [   0.00

In [37]:
save_hybrid=False
single_cls = False

In [38]:
conf_thres = 0.001
iou_thres = 0.6

In [39]:
lb = [targets[targets[:, 0] == i, 1:] for i in range(nb)] if save_hybrid else []  # for autolabelling

In [40]:
lb

[]

In [41]:
out = non_max_suppression(out, conf_thres, iou_thres, labels=lb, multi_label=True, agnostic=single_cls)

In [52]:
for si,pred in enumerate(out):
    labels = targets[targets[:, 0] == si, 1:]
    nl = len(labels)
    tcls = labels[:, 0].tolist()
    path, shape = Path(paths[si]), shapes[si][0]
    seen += 1
    predn = pred.clone()
    scale_coords(img[si].shape[1:], predn[:, :4], shape, shapes[si][1])  # native-space pred
    tbox = xywh2xyxy(labels[:, 1:5])
    scale_coords(img[si].shape[1:], tbox, shape, shapes[si][1])  #
    labelsn = torch.cat((labels[:, 0:1], tbox), 1)
   
    break
    

torch.Size([300, 6])


In [53]:
labels = targets[targets[:, 0] == si, 1:]

In [78]:
labels.shape

torch.Size([60, 5])

In [95]:
correct = process_batch(predn, labelsn, iouv)

In [96]:
correct

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [115]:
correct[296]

tensor([False, False, False, False, False, False, False, False, False, False])

In [101]:
confusion_matrix

In [84]:
detections = predn
labels = labelsn

In [130]:
correct = torch.zeros(detections.shape[0], iouv.shape[0], dtype=torch.bool, device=iouv.device)

In [131]:
iou = box_iou(labels[:, 1:], detections[:, :4])

In [ ]:
x = torch.where((iou >= iouv[0]) & (labels[:, 0:1] == detections[:, 5]))  # IoU above threshold and classes match*

In [134]:
if x[0].shape[0]:
    matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()  # [label, detection, iou]

In [136]:
matches.shape

(49, 3)

In [137]:
if x[0].shape[0]:
    matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()  # [label, detection, iou]
    if x[0].shape[0] > 1:
            matches = matches[matches[:, 2].argsort()[::-1]]
            matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
            matches = matches[np.unique(matches[:, 0], return_index=True)[1]]

In [138]:
matches.shape

(24, 3)

In [113]:
matches = torch.Tensor(matches).to(iouv.device)

In [114]:
correct[matches[:, 1].long()] = matches[:, 2:3] >= iouv

In [118]:
indices = torch.nonzero(correct, as_tuple=False)

print(indices)



tensor([[ 18,   0],
        [ 18,   1],
        [ 18,   2],
        [ 18,   3],
        [ 18,   4],
        [ 18,   5],
        [ 18,   6],
        [ 18,   7],
        [ 18,   8],
        [ 22,   0],
        [ 22,   1],
        [ 22,   2],
        [ 22,   3],
        [ 22,   4],
        [ 22,   5],
        [ 22,   6],
        [ 31,   0],
        [ 31,   1],
        [ 31,   2],
        [ 31,   3],
        [ 31,   4],
        [ 31,   5],
        [ 31,   6],
        [ 34,   0],
        [ 34,   1],
        [ 34,   2],
        [ 34,   3],
        [ 34,   4],
        [ 34,   5],
        [ 34,   6],
        [ 38,   0],
        [ 38,   1],
        [ 38,   2],
        [ 38,   3],
        [ 38,   4],
        [ 38,   5],
        [ 38,   6],
        [ 38,   7],
        [ 43,   0],
        [ 43,   1],
        [ 43,   2],
        [ 43,   3],
        [ 71,   0],
        [ 71,   1],
        [ 71,   2],
        [ 71,   3],
        [ 73,   0],
        [ 73,   1],
        [ 75,   0],
        [ 75,   1],


In [119]:
print(matches)

tensor([[  1.00000,  31.00000,   0.82413],
        [  2.00000,  22.00000,   0.80298],
        [  3.00000,  34.00000,   0.81166],
        [  4.00000,  43.00000,   0.69488],
        [  5.00000,  18.00000,   0.91025],
        [  6.00000,  38.00000,   0.86246],
        [  7.00000,  73.00000,   0.59322],
        [  8.00000, 145.00000,   0.54356],
        [  9.00000, 136.00000,   0.75438],
        [ 10.00000,  94.00000,   0.78315],
        [ 11.00000,  88.00000,   0.73774],
        [ 13.00000,  71.00000,   0.69170],
        [ 14.00000, 177.00000,   0.62261],
        [ 15.00000, 137.00000,   0.85195],
        [ 16.00000, 111.00000,   0.65725],
        [ 17.00000, 112.00000,   0.55810],
        [ 18.00000,  89.00000,   0.71083],
        [ 26.00000,  75.00000,   0.71600],
        [ 27.00000, 123.00000,   0.92472],
        [ 53.00000, 246.00000,   0.75206],
        [ 54.00000, 226.00000,   0.87520],
        [ 55.00000, 244.00000,   0.74083],
        [ 56.00000, 237.00000,   0.71806],
        [ 5

In [123]:
matches[:, 2:3] 

tensor([[0.82413],
        [0.80298],
        [0.81166],
        [0.69488],
        [0.91025],
        [0.86246],
        [0.59322],
        [0.54356],
        [0.75438],
        [0.78315],
        [0.73774],
        [0.69170],
        [0.62261],
        [0.85195],
        [0.65725],
        [0.55810],
        [0.71083],
        [0.71600],
        [0.92472],
        [0.75206],
        [0.87520],
        [0.74083],
        [0.71806],
        [0.70454]])

In [124]:
iouv

tensor([0.50000, 0.55000, 0.60000, 0.65000, 0.70000, 0.75000, 0.80000, 0.85000, 0.90000, 0.95000])

In [125]:
correct

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [126]:
matches[:, 2:3] >= iouv

tensor([[ True,  True,  True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True,  True,  True, False, False],
        [ True,  True, False, False, False, False, False, False, False, False],
        [ True, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False, False, False],
        [ True,  True,  True, False, Fal

In [127]:
matches[:, 1].long()

tensor([ 31,  22,  34,  43,  18,  38,  73, 145, 136,  94,  88,  71, 177, 137, 111, 112,  89,  75, 123, 246, 226, 244, 237, 127])

In [129]:
correct[31]

tensor([ True,  True,  True,  True,  True,  True,  True, False, False, False])